<a href="https://colab.research.google.com/github/Tanguyvans/StGeorge/blob/main/StGeorge_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import sys
import urllib
import os

import pandas as pd
import csv
from google.colab import drive

#1. Data Loader

In [28]:
drive.mount('/content/gdrive')
drive_data_folder = '/content/gdrive/MyDrive/george_dataset/'

if os.path.exists(drive_data_folder) == False:
  os.mkdir('/content/gdrive/MyDrive/george_dataset')
  os.mkdir('/content/gdrive/MyDrive/george_dataset/george')
  os.mkdir('/content/gdrive/MyDrive/george_dataset/non_george')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
george = pd.read_csv('/content/gdrive/MyDrive/test_assignment_cv/georges.csv', header = None)

for index, row in george.iterrows(): 
  urllib.request.urlretrieve(row[0], '/content/gdrive/MyDrive/george_dataset/george/'+str(index)+'.jpg')